## Deliverable 2. Create a Customer Travel Destinations Map.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [18]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Karratha,AU,-20.7377,116.8463,82.0,75,92,12.21,light rain
1,1,Puerto Ayora,EC,-0.7393,-90.3518,86.0,62,20,14.97,few clouds
2,2,Exeter,GB,50.7236,-3.5275,42.8,81,20,3.44,few clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,42.8,70,40,32.21,scattered clouds
4,4,Ilulissat,GL,69.2167,-51.1000,33.8,80,75,10.36,snow


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Karratha,AU,-20.7377,116.8463,82.00,75,92,12.21,light rain
1,1,Puerto Ayora,EC,-0.7393,-90.3518,86.00,62,20,14.97,few clouds
13,13,Rikitea,PF,-23.1203,-134.9692,76.46,64,16,2.44,few clouds
16,16,Tautira,PF,-17.7333,-149.1500,87.80,66,40,5.75,scattered clouds
17,17,Vanimo,PG,-2.6741,141.3028,78.13,87,100,3.53,overcast clouds
23,23,Bitkine,TD,11.9801,18.2138,88.66,11,45,6.13,scattered clouds
24,24,Padilla,CO,3.2204,-76.3139,82.40,57,75,16.11,broken clouds
28,28,Wajid,SO,3.8093,43.2471,81.03,64,100,19.13,overcast clouds
29,29,Albany,US,42.6001,-73.9662,75.20,35,20,4.61,few clouds
31,31,Arraial Do Cabo,BR,-22.9661,-42.0278,75.20,83,40,4.61,scattered clouds


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                219
City                   219
Country                217
Lat                    219
Lng                    219
Max Temp               219
Humidity               219
Cloudiness             219
Wind Speed             219
Weather Description    219
dtype: int64

In [24]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                217
City                   217
Country                217
Lat                    217
Lng                    217
Max Temp               217
Humidity               217
Cloudiness             217
Wind Speed             217
Weather Description    217
dtype: int64

In [26]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Karratha,AU,82.00,light rain,-20.7377,116.8463,
1,Puerto Ayora,EC,86.00,few clouds,-0.7393,-90.3518,
13,Rikitea,PF,76.46,few clouds,-23.1203,-134.9692,
16,Tautira,PF,87.80,scattered clouds,-17.7333,-149.1500,
17,Vanimo,PG,78.13,overcast clouds,-2.6741,141.3028,
23,Bitkine,TD,88.66,scattered clouds,11.9801,18.2138,
24,Padilla,CO,82.40,broken clouds,3.2204,-76.3139,
28,Wajid,SO,81.03,overcast clouds,3.8093,43.2471,
29,Albany,US,75.20,few clouds,42.6001,-73.9662,
31,Arraial Do Cabo,BR,75.20,scattered clouds,-22.9661,-42.0278,


In [37]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [42]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()


In [43]:
#Check dataframe
clean_hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Karratha,AU,82.00,light rain,-20.7377,116.8463,ibis Styles Karratha
1,Puerto Ayora,EC,86.00,few clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
13,Rikitea,PF,76.46,few clouds,-23.1203,-134.9692,Pension Maro'i
16,Tautira,PF,87.80,scattered clouds,-17.7333,-149.1500,Pension Oaoa
17,Vanimo,PG,78.13,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
...,...,...,...,...,...,...,...
663,Puerto Ayacucho,VE,78.17,light rain,5.6639,-67.6236,Posada Turistica Manapiare
664,Popondetta,PG,76.68,overcast clouds,-8.7537,148.2534,Timos Transit House
667,Mercedes,AR,75.99,moderate rain,-34.6515,-59.4307,Posada del Inti Centro
668,Bubaque,GW,76.69,clear sky,11.2833,-15.8333,"Casa Dora, Bubaque"


In [45]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [49]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [52]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))